# Importer les Bibliothèques Python
* On utilise `requests` pour l'aspiration d'une ressource web.
* On utilise `csv` pour la lecture des fichiers CSV.
* On utilise `tarfile` pour la décompression des archives TAR.
* On utilise `os` et `glob` pour la gestion des fichiers et des dossiers de l'environnement.
* On utilise le module `etree` de la bibliothèque Python `lxml` pour le traitement des fichiers XML.

Les six bibliothèques Python sont installées par défaut sur Google Colab. Néanmoins, si vous voulez utiliser le même code sur un Jupyter Notebook local ou dans un compilateur Python, veuillez installer ces bibliothèques en utilisant pip.

In [1]:
import requests
import csv
import tarfile
import os
import glob
from lxml import etree

#Aspiration des liens de téléchargement de PubMed Central
* Dans le dump de PubMed Central, il y a un fichier CSV qui permet d'identifier le lien de téléchargement d'un article en se basant sur son identifiant PubMed Central.
* Le lien ménera vers une archive TAR.GZ qui contient:
  * Le fichier PDF de l'article
  * Les figures de l'article
  * Le fichier NXML de l'article qui est un fichier XML représentant l'ensemble de l'article.

In [2]:
url = "https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_file_list.csv"
response = requests.get(url)

# Téléchargement et décompression de l'archive TAR.GZ
* La fonction du téléchargement et de la décompression des archives TAR.GZ permettra de:
  * Télécharger le fichier TAR.GZ
  * Décompresser l'archive TAR.GZ et mettre les fichiers qu'elle contient dans un dossier spécifique
  * Supprimer le fichier TAR.GZ.

* On a deux éditions de cette fonction:
  * **Une qui utilise le Shell Scripting:** C'est une édition qui fonctionne plus rapidement. Néanmoins, elle ne fonctionne pas si elle est mise dans un compilateur Python.
  * **Une qui utilise des bibliothèques Python:** C'est une édition plus lente. Mais, elle fonctionne sur n'importe quel compilateur Python à condition que les bibliothèques Python sont installées.

In [3]:
#Version avec un script Shell
def download_file(pmcid):
  for row in csv.reader(response.text.splitlines()):
    # Read and parse the CSV file
    if "PMC"+pmcid+".tar.gz" in row[0]:
        !wget https://ftp.ncbi.nlm.nih.gov/pub/pmc/{row[0]}
        !tar -xvzf {"PMC"+pmcid+".tar.gz"}
        !rm {"PMC"+pmcid+".tar.gz"}

In [ ]:
#Version avec des bibliothèques Python
def download_file(pmcid):
  for row in csv.reader(response.text.splitlines()):
    # Read and parse the CSV file
    if "PMC"+pmcid+".tar.gz" in row[0]:
      file_url = "https://ftp.ncbi.nlm.nih.gov/pub/pmc/" + row[0]
      file_name = "PMC"+pmcid+".tar.gz"

      # Download the file
      file_response = requests.get(file_url, stream=True)
      file_response.raise_for_status()

      with open(file_name, "wb") as f:
        for chunk in file_response.iter_content(chunk_size=8192):
          f.write(chunk)

      # Extract the tar.gz file
      with tarfile.open(file_name, "r:gz") as tar:
         tar.extractall()

      # Remove the downloaded tar.gz file
      os.remove(file_name)

In [4]:
download_file("11208141")

--2025-03-15 15:16:59--  https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/88/de/PMC11208141.tar.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4412891 (4.2M) [application/x-gzip]
Saving to: ‘PMC11208141.tar.gz’

PMC11208141.tar.gz  100%[===================>]   4.21M  20.8MB/s    in 0.2s    

2025-03-15 15:16:59 (20.8 MB/s) - ‘PMC11208141.tar.gz’ saved [4412891/4412891]

PMC11208141/
PMC11208141/41586_2024_7335_Article_IEq4.gif
PMC11208141/41586_2024_7335_Article_IEq6.gif
PMC11208141/41586_2024_7335_Article_IEq12.gif
PMC11208141/41586_2024_7335_Article_Equ1.gif
PMC11208141/41586_2024_7335_Fig1_HTML.gif
PMC11208141/41586_2024_7335_Fig3_HTML.jpg
PMC11208141/41586_2024_7335_Article_IEq8.gif
PMC11208141/41586_2024_7335_Article_Equ2.gif
PMC11208141/41586_2024_7335_Fig3_HTML.gif
PMC1120814

# Exploration du fichier NXML
* On utilise `lxml.etree` pour retrouver l'arborescence des balises XML dans un fichier NXML.
* Pour expliquer la structure d'un fichier XML, on peut dire que l'élément est l'entité élémentaire du fichier. Chaque élément a le format suivant:
`<balise attribut1="valeur_attribut1">contenu</balise>`.
* La balise XML permet d'identifier la nature de l'entité.
* Les attributs permettent de définir les paramètres de la balise.
* Le contenu est le texte de l'entité.
* Un contenu peut contenir une balise fille. Ceci explique la génération de l'arborescence des balises XML sous la forme d'un arbre ramifié.
* En voyant cet exemple, on retrouve que:
  * Le parent commun de toutes les balises est "article".
  * La balise "front" contient des données bibliographiques de l'article à savoir le titre (`article-title`), le journal (`journal-title`), les co-auteurs (`contrib`) et les affiliations (`aff`).
  * La balise "body" contient le texte intégral de l'article à savoir les paragraphes (`p`), les références (`xref`), les tableaux (`table-wrap`) et les figures (`fig`).



In [5]:
def print_tree(element, indent=0):
    tag_name = getattr(element, 'tag', getattr(element, 'name', 'Unknown'))  # Handle both lxml/ElementTree and BeautifulSoup
    print("  " * indent + str(tag_name))

    for child in list(element):  # Ensure iterable child elements
        print_tree(child, indent + 1)

def parse_nxml(pmc_id, base_dir="."):
    dir_path = os.path.join(base_dir, "PMC"+pmc_id)

    if not os.path.isdir(dir_path):
        raise FileNotFoundError(f"Directory not found: {dir_path}")

    # Cherche un fichier .nxml dans le dossier
    nxml_files = glob.glob(os.path.join(dir_path, "*.nxml"))

    if not nxml_files:
        raise FileNotFoundError(f"No .nxml file found in {dir_path}")

    file_path = nxml_files[0]  # Prend le premier fichier trouvé (unique dans le dossier)

    tree = etree.parse(file_path)
    root = tree.getroot()
    print_tree(root)
    return root

# Usage
pmc_id = "11208141"
root = parse_nxml(pmc_id, base_dir=".")

article
  <cyfunction ProcessingInstruction at 0x78f2b6817b90>
  processing-meta
    restricted-by
  front
    journal-meta
      journal-id
      journal-id
      journal-title-group
        journal-title
      issn
      issn
      publisher
        publisher-name
        publisher-loc
    article-meta
      article-id
      article-id
      article-id
      article-id
      article-categories
        subj-group
          subject
      title-group
        article-title
      contrib-group
        contrib
          collab
            contrib-group
              contrib
                contrib-id
                name
                  surname
                  given-names
                address
                  email
                xref
              contrib
                name
                  surname
                  given-names
                xref
              contrib
                name
                  surname
                  given-names
                xref
          

# Extraction des données du fichier NXML
* Après l'exploration de l'arborescence (`root`), on peut utiliser cette taxonomie pour extraire des données spécifiques comme le titre de l'article, les paragraphes de l'article ou la légende d'une figure.
* On utilise `find` si on cherche la première apparition d'une balise particulière.
* On utilise `findall` si on cherche toutes les occurrences d'une balise particulière. Puis, on les traite en utilisant une boucle Pour.

In [6]:
# Extraire le titre du travail
title = root.find(".//article-title")
if title is not None:
  print("Titre :", title.text)

Titre : Scaling neural machine translation to 200 languages


In [7]:
# Extraire tous les paragraphes
paragraphs = root.findall(".//p")
for i, para in enumerate(paragraphs, 1):
  print(f"Paragraphe {i} : {para.text}")

Paragraphe 1 : The development of neural techniques has opened up new avenues for research in machine translation. Today, neural machine translation (NMT) systems can leverage highly multilingual capacities and even perform zero-shot translation, delivering promising results in terms of language coverage and quality. However, scaling quality NMT requires large volumes of parallel bilingual data, which are not equally available for the 7,000+ languages in the world
Paragraphe 2 : Scaling neural machine translation to 200 languages is achieved by No Language Left Behind, a single massively multilingual model that leverages transfer learning across languages.
Paragraphe 3 : The recent advent of neural machine translation (NMT) has pushed translation technologies to new frontiers, but its benefits are unevenly distributed
Paragraphe 4 : It has now been widely acknowledged that multilingual models have demonstrated promising performance improvement over bilingual models
Paragraphe 5 : First

In [8]:
# Extraire le titre de chaque figure
figures = root.findall(".//fig/caption/title")
for i, para in enumerate(figures, 1):
  print(f"Figure {i} : {para.text}")

Figure 1 : Cosine similarity scores between languages at different layers of the encoder–decoder architecture.
Figure 2 : Quality of FLORES-200.
Figure 3 : Correlations between aggregated human quality scores and automated metrics.
Figure 4 : Architecture of the LASER3 teacher-student approach.
Figure 5 : Illustration of EOM (panel c) in contrast to overall dropout (panel b) for MoE layers.
